## Get The text from the PDF

In [1]:
%pip install pdfplumber pytesseract pdf2image

     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.8 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.8 kB ? eta -:--:--
     ------------------ -------------------- 20.5/42.8 kB 93.1 kB/s eta 0:00:01
     ------------------ -------------------- 20.5/42.8 kB 93.1 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.8 kB 119.1 kB/s eta 0:00:01
     -------------------------------------- 42.8/42.8 kB 138.4 kB/s eta 0:00:00
  Using cached pypdfium2-4.30.1-py3-none-win_amd64.whl.metadata (48 kB)
  Using cached cryptography-44.0.2-cp39-abi3-win_amd64.whl.metadata (5.7 kB)
   ---------------------------------------- 0.0/60.2 kB ? eta -:--:--
   ---------------------------------- ----- 51.2/60.2 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 60.2/60.2 kB 793.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
    ------------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path

In [4]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        # Try direct text extraction
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text

        if text.strip():
            return text.strip()
    except Exception as e:
        print(f"Direct text extraction failed: {e}")

    # Fallback to OCR for image-based PDFs
    print("Falling back to OCR for image-based PDF.")
    try:
        images = convert_from_path(pdf_path)
        for image in images:
            page_text = pytesseract.image_to_string(image)
            text += page_text + "\n"
    except Exception as e:
        print(f"OCR failed: {e}")

    return text.strip()

In [17]:
pdf_path = "Resume.pdf"
resume_text = extract_text_from_pdf(pdf_path)

print("\nExtracted Text from PDF:")
print(resume_text)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox



Extracted Text from PDF:
MAX JOHNSON
Mumbai, India | maxjohnson1209@gmail.com |+91-7039314886 | linkedin.com/in/maxjohnson-1209/
EXPERIENCE
Oracle Financial Services Software Limited | Associate Consultant Mumbai, MH, India| July 2023 - Present
• Creating tailored pitch decks, brochures, and solution overviews for client engagements, ensuring alignment with market
trends and customer requirements.
• Collaborating with cross-functional teams to define and execute GTM strategies, including market research, segmentation.
• Skills- OCI, Python, Gen-AI, Agile Methodology, Content Packaging, Confluence, GTM Strategy.
Fractal | Partner & Alliance Intern Mumbai, MH, India | Aug 2022 – March 2023
• Worked as web designer & Fractal core team to create Google Cloud microsite focusing Fractal & Google Cloud 360* strategic
partnership to generate inbound leads.
• Executed GTM Strategy for Google Cloud India Business and Demand Generation Plans.
• Worked jointly with the marketing team to create a 

## Set Google GenerativeAI Api Key

In [18]:
%pip install google.generativeai python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-flash")

c:\Users\Andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
response = model.generate_content("Does RCB gona win IPL 2025")


In [14]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Whether the Royal Challengers Bangalore (RCB) will win IPL 2025 is impossible to predict with certainty.  The IPL is a highly competitive tournament where many factors influence the outcome, including player form, injuries, team chemistry, and luck.  While RCB has a strong fanbase and a talented roster, predicting a win two years out is pure speculation.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.15194688345256605
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 12,
        "candidates_token_count": 76,
        "total_token_count": 88
      },
      "model_version": "gemini-1.5-flash"
    }),
)


In [15]:
print(response.text)

Whether the Royal Challengers Bangalore (RCB) will win IPL 2025 is impossible to predict with certainty.  The IPL is a highly competitive tournament where many factors influence the outcome, including player form, injuries, team chemistry, and luck.  While RCB has a strong fanbase and a talented roster, predicting a win two years out is pure speculation.



## Resume Analysis

In [19]:
def analyze_resume(resume_text, job_description=None):
    if not resume_text:
        return {"error": "Resume text is required for analysis."}
    
    model = genai.GenerativeModel("gemini-1.5-flash")
    
    base_prompt = f"""
    You are an experienced HR with Technical Experience in the field of any one job role from Data Science, Data Analyst, DevOPS, Machine Learning Engineer, Prompt Engineer, AI Engineer, Full Stack Web Development, Big Data Engineering, Marketing Analyst, Human Resource Manager, Software Developer your task is to review the provided resume.
    Please share your professional evaluation on whether the candidate's profile aligns with the role.ALso mention Skills he already have and siggest some skills to imorve his resume , alos suggest some course he might take to improve the skills.Highlight the strengths and weaknesses.

    Resume:
    {resume_text}
    """

    if job_description:
        base_prompt += f"""
        Additionally, compare this resume to the following job description:
        
        Job Description:
        {job_description}
        
        Highlight the strengths and weaknesses of the applicant in relation to the specified job requirements.
        """

    response = model.generate_content(base_prompt)

    analysis = response.text.strip()
    return analysis

In [20]:
print(analyze_resume(resume_text))


## Resume Evaluation for Max Johnson

**Role Focus:**  To best evaluate Max's resume, I will assume we are considering roles that leverage his experience in **Marketing/GTM Strategy with a focus on AI/ML applications**. This aligns with his experience at Oracle and Fractal, as well as his project portfolio.  My expertise as an experienced HR professional with a technical background in **Data Science** informs this assessment.

**Strengths:**

* **Relevant Experience:** Max's internships and current role showcase experience in GTM strategy, market research, client engagement, and project management, all highly relevant to marketing roles with a technical component.
* **Strong Technical Skills:**  His skills in Python, various cloud platforms (GCP, OCI), and AI/ML tools (LangChain, OpenAI, Hugging Face) are significant assets.  The Oracle certifications are impressive and demonstrate a commitment to learning.
* **Impressive Projects:** The "Document Genie" and "FinSight" projects clearly